In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import sys

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
sys.path.insert(0, SOURCE_DIR)

In [3]:
import tensorflow as tf
import malaya_speech
import malaya_speech.train
from malaya_speech.train.model import tacotron2_nvidia as tacotron2
import numpy as np






The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.





In [4]:
input_ids = tf.placeholder(tf.int32, [None, None])
input_lengths = tf.placeholder(tf.int32, [None])
mel_outputs = tf.placeholder(tf.float32, [None, None, 80])
mel_lengths = tf.placeholder(tf.int32, [None])

In [5]:
model = tacotron2.Model([input_ids, input_lengths], [mel_outputs, mel_lengths], 20)



Instructions for updating:
Use `tf.keras.layers.Conv1D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).
Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for u

In [6]:
model.encoder_logits

{'outputs': <tf.Tensor 'tacotron2_encoder/concat:0' shape=(?, ?, 512) dtype=float32>,
 'src_length': <tf.Tensor 'tacotron2_encoder/floordiv_2:0' shape=(?,) dtype=int32>}

In [7]:
model.decoder_logits

{'outputs': [<tf.Tensor 'tacotron_2_decoder/decoder_1/output_proj/BiasAdd:0' shape=(?, ?, 80) dtype=float32>,
  <tf.Tensor 'tacotron_2_decoder/add:0' shape=(?, ?, 80) dtype=float32>,
  <tf.Tensor 'tacotron_2_decoder/transpose_1:0' shape=(?, ?, ?) dtype=float32>,
  <tf.Tensor 'tacotron_2_decoder/Sigmoid:0' shape=(?, ?, 1) dtype=float32>,
  <tf.Tensor 'tacotron_2_decoder/decoder/while/Exit_15:0' shape=(?,) dtype=int32>,
  <tf.Tensor 'tacotron_2_decoder/zeros:0' shape=(?, ?, ?) dtype=float32>],
 'stop_token_prediction': <tf.Tensor 'tacotron_2_decoder/decoder_1/stop_token_proj/BiasAdd:0' shape=(?, ?, 1) dtype=float32>}

In [8]:
guided = tacotron2.generate_guided_attention(mel_lengths[0], input_lengths[0])
guided = tf.expand_dims(guided, 0)
guided

<tf.Tensor 'ExpandDims:0' shape=(1, ?, ?) dtype=float32>

In [9]:
from malaya_speech.train.loss import calculate_2d_loss, calculate_3d_loss

In [10]:
r = model.decoder_logits['outputs']
decoder_output, post_mel_outputs, alignment_histories, stop_token_predictions, _, _ = r
stop_token_predictions = stop_token_predictions[:,:,0]

In [11]:
binary_crossentropy = tf.keras.losses.BinaryCrossentropy(
    from_logits=True
)
mse = tf.keras.losses.MeanSquaredError()
mae = tf.keras.losses.MeanAbsoluteError()
mel_loss_before = calculate_3d_loss(
    mel_outputs, decoder_output, loss_fn=mae
)
mel_loss_after = calculate_3d_loss(
    mel_outputs, post_mel_outputs, loss_fn=mae
)
max_mel_length = tf.reduce_max(mel_lengths)
stop_gts = tf.expand_dims(
    tf.range(tf.reduce_max(max_mel_length), dtype=tf.int32), 0
)
stop_gts = tf.tile(
    stop_gts, [tf.shape(mel_lengths)[0], 1]
)
stop_gts = tf.cast(
    tf.math.greater_equal(stop_gts, tf.expand_dims(mel_lengths, 1)),
    tf.float32,
)
stop_token_loss = calculate_2d_loss(
    stop_gts, stop_token_predictions, loss_fn=binary_crossentropy
)
attention_masks = tf.cast(
    tf.math.not_equal(guided, -1.0), tf.float32
)
loss_att = tf.reduce_sum(
    tf.abs(alignment_histories * guided) * attention_masks,
    axis=[1, 2],
)
loss_att /= tf.reduce_sum(attention_masks, axis=[1, 2])

In [12]:
loss_att

<tf.Tensor 'truediv_3:0' shape=(?,) dtype=float32>

In [13]:
loss = stop_token_loss + mel_loss_before + mel_loss_after + loss_att

In [14]:
stop_token_loss, mel_loss_before, mel_loss_after, loss_att

(<tf.Tensor 'binary_crossentropy/weighted_loss/value:0' shape=() dtype=float32>,
 <tf.Tensor 'mean_absolute_error/weighted_loss/value:0' shape=() dtype=float32>,
 <tf.Tensor 'mean_absolute_error_1/weighted_loss/value:0' shape=() dtype=float32>,
 <tf.Tensor 'truediv_3:0' shape=(?,) dtype=float32>)

In [15]:
# r = model.inference(input_ids, input_lengths, speaker_ids)
# r

In [16]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [17]:
sess.run(decoder_output, feed_dict = {input_ids: [[1, 2, 3, 4, 5, 6, 7, 8, 9]],
                        input_lengths: [9],
                        mel_outputs: np.random.normal(size=(1, 50, 80)).astype(np.float32),
                        mel_lengths: [50]}).shape

(1, 50, 80)

In [18]:
sess.run(alignment_histories, feed_dict = {input_ids: [[1, 2, 3, 4, 5, 6, 7, 8, 9]],
                        input_lengths: [9],
                        mel_outputs: np.random.normal(size=(1, 50, 80)).astype(np.float32),
                        mel_lengths: [50]}).shape

(1, 9, 50)

In [19]:
sess.run(r, feed_dict = {input_ids: [[1, 2, 3, 4, 5, 6, 7, 8, 9]],
                        input_lengths: [9],
                        mel_outputs: np.random.normal(size=(1, 50, 80)).astype(np.float32),
                        mel_lengths: [50]})

[array([[[-0.07404951, -0.07576215,  0.05768657, ...,  0.19319254,
           0.09480648, -0.05262316],
         [-0.08680883, -0.08005451,  0.04620728, ...,  0.19497743,
           0.09577974, -0.06322172],
         [-0.10269383, -0.06106855,  0.05190963, ...,  0.20084828,
           0.10037264, -0.07901663],
         ...,
         [-0.07012279, -0.0481539 , -0.08297201, ...,  0.32089975,
          -0.06296035, -0.10910055],
         [-0.14776605, -0.04074803, -0.11973462, ...,  0.3040336 ,
          -0.03534711, -0.0999113 ],
         [-0.09117725, -0.05362306, -0.06122065, ...,  0.29135406,
           0.00580565, -0.12459618]]], dtype=float32),
 array([[[-2.0291548 , -0.07576215,  0.05768657, ...,  0.19319254,
           0.09480648, -0.5988778 ],
         [-0.08680883, -0.08005451,  0.04620728, ..., -2.7248313 ,
           0.09577974,  0.08272941],
         [-0.10269383, -0.06106855,  0.05190963, ...,  0.02408612,
           1.4484916 , -0.07901663],
         ...,
         [-0.81164

In [20]:
sess.run(loss, feed_dict = {input_ids: [[1, 2, 3, 4, 5, 6, 7, 8, 9]],
                        input_lengths: [9],
                        mel_outputs: np.random.normal(size=(1, 50, 80)).astype(np.float32),
                        mel_lengths: [50]})

array([3.1633177], dtype=float32)

In [22]:
sess.run(loss, feed_dict = {input_ids: [[1, 2, 3, 4, 5, 6, 7, 8, 9],
                                       [1, 2, 3, 4, 5, 6, 7, 8, 9]],
                        input_lengths: [9, 9],
                        mel_outputs: np.random.normal(size=(2, 50, 80)).astype(np.float32),
                        mel_lengths: [50, 50]})

array([3.1496131, 3.1502788], dtype=float32)

In [24]:
sess.run(guided, feed_dict = {input_ids: [[1, 2, 3, 4, 5, 6, 7, 8, 9]],
                        input_lengths: [9],
                        mel_outputs: np.random.normal(size=(1, 50, 80)).astype(np.float32),
                        mel_lengths: [50]}).shape

(1, 9, 50)